In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf

rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

# set Dropout
do = 0.3

if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss
# # remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# feat = feat[mask]
# API model, if use RNN, first two rnn layer must return_sequences=True


Using TensorFlow backend.


In [2]:
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(dropout_rate)(net_speech)

    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_speech) for name in target_names]

    model = Model(input_speech, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model

#def main(alpha, beta, gamma):
#model = api_model(0.1, 0.5, 0.4)
#model = api_model(1, 1, 1)
#model.summary()
# main function for for LOSO
# main function for for LOSO


In [3]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:7869]],
                      vad[:7869].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[7869:]], vad[7869:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('speech_8layer.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 7s 1ms/step - loss: 1.8771 - v_loss: 0.8920 - a_loss: 0.4417 - d_loss: 0.5430 - v_ccc: 0.1077 - a_ccc: 0.5582 - d_ccc: 0.4570 - val_loss: 2.5757 - val_v_loss: 0.8890 - val_a_loss: 0.8649 - val_d_loss: 0.8175 - val_v_ccc: 0.1095 - val_a_ccc: 0.1361 - val_d_ccc: 0.1788
Epoch 2/50
6295/6295 [==============================] - 3s 441us/step - loss: 1.5930 - v_loss: 0.7447 - a_loss: 0.3740 - d_loss: 0.4752 - v_ccc: 0.2558 - a_ccc: 0.6259 - d_ccc: 0.5253 - val_loss: 2.0650 - val_v_loss: 0.8607 - val_a_loss: 0.5935 - val_d_loss: 0.6153 - val_v_ccc: 0.1395 - val_a_ccc: 0.4103 - val_d_ccc: 0.3852
Epoch 3/50
6295/6295 [==============================] - 3s 468us/step - loss: 1.5546 - v_loss: 0.7123 - a_loss: 0.3693 - d_loss: 0.4728 - v_ccc: 0.2876 - a_ccc: 0.6306 - d_ccc: 0.5272 - val_loss: 1.9745 - val_v_loss: 0.8

Epoch 6/50
6295/6295 [==============================] - 3s 491us/step - loss: 1.4975 - v_loss: 0.6656 - a_loss: 0.3626 - d_loss: 0.4692 - v_ccc: 0.3343 - a_ccc: 0.6374 - d_ccc: 0.5308 - val_loss: 2.0135 - val_v_loss: 0.8549 - val_a_loss: 0.5440 - val_d_loss: 0.6138 - val_v_ccc: 0.1433 - val_a_ccc: 0.4583 - val_d_ccc: 0.3849
Epoch 7/50
6295/6295 [==============================] - 3s 501us/step - loss: 1.4873 - v_loss: 0.6535 - a_loss: 0.3631 - d_loss: 0.4706 - v_ccc: 0.3463 - a_ccc: 0.6370 - d_ccc: 0.5294 - val_loss: 2.0419 - val_v_loss: 0.8598 - val_a_loss: 0.5646 - val_d_loss: 0.6158 - val_v_ccc: 0.1386 - val_a_ccc: 0.4371 - val_d_ccc: 0.3824
Epoch 8/50
6295/6295 [==============================] - 4s 614us/step - loss: 1.4842 - v_loss: 0.6556 - a_loss: 0.3628 - d_loss: 0.4656 - v_ccc: 0.3444 - a_ccc: 0.6371 - d_ccc: 0.5344 - val_loss: 2.0066 - val_v_loss: 0.8608 - val_a_loss: 0.5399 - val_d_loss: 0.6061 - val_v_ccc: 0.1384 - val_a_ccc: 0.4631 - val_d_ccc: 0.3920
Epoch 9/50
6295/6295 [

6295/6295 [==============================] - 3s 474us/step - loss: 1.7154 - v_loss: 0.8783 - a_loss: 0.3468 - d_loss: 0.4906 - v_ccc: 0.1218 - a_ccc: 0.6533 - d_ccc: 0.5096 - val_loss: 1.9799 - val_v_loss: 0.8752 - val_a_loss: 0.5362 - val_d_loss: 0.5720 - val_v_ccc: 0.1243 - val_a_ccc: 0.4674 - val_d_ccc: 0.4285
Epoch 3/50
6295/6295 [==============================] - 3s 479us/step - loss: 1.5955 - v_loss: 0.7627 - a_loss: 0.3588 - d_loss: 0.4740 - v_ccc: 0.2370 - a_ccc: 0.6413 - d_ccc: 0.5261 - val_loss: 1.9594 - val_v_loss: 0.8643 - val_a_loss: 0.5217 - val_d_loss: 0.5778 - val_v_ccc: 0.1356 - val_a_ccc: 0.4828 - val_d_ccc: 0.4222
Epoch 4/50
6295/6295 [==============================] - 3s 473us/step - loss: 1.5438 - v_loss: 0.7094 - a_loss: 0.3611 - d_loss: 0.4736 - v_ccc: 0.2904 - a_ccc: 0.6391 - d_ccc: 0.5267 - val_loss: 1.9771 - val_v_loss: 0.8729 - val_a_loss: 0.5167 - val_d_loss: 0.5903 - val_v_ccc: 0.1260 - val_a_ccc: 0.4876 - val_d_ccc: 0.4093
Epoch 5/50
6295/6295 [===========

2170/2170 [==============================] - 0s 165us/step
[0.09322760999202728, 0.548517107963562, 0.43229737877845764]
4
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 8s 1ms/step - loss: 1.9388 - v_loss: 0.9664 - a_loss: 0.4241 - d_loss: 0.5476 - v_ccc: 0.0335 - a_ccc: 0.5757 - d_ccc: 0.4520 - val_loss: 2.4334 - val_v_loss: 0.8229 - val_a_loss: 0.8222 - val_d_loss: 0.7818 - val_v_ccc: 0.1760 - val_a_ccc: 0.1771 - val_d_ccc: 0.2135
Epoch 2/50
6295/6295 [==============================] - 3s 507us/step - loss: 1.6072 - v_loss: 0.7668 - a_loss: 0.3648 - d_loss: 0.4754 - v_ccc: 0.2332 - a_ccc: 0.6349 - d_ccc: 0.5247 - val_loss: 2.0337 - val_v_loss: 0.8599 - val_a_loss: 0.5815 - val_d_loss: 0.5990 - val_v_ccc: 0.1405 - val_a_ccc: 0.4233 - val_d_ccc: 0.4025
Epoch 3/50
6295/6295 [==============================] - 3s 519us/step - loss: 1.5544 - v_loss: 0.7256 - a_loss: 0.3610 - d_loss: 0.4679 - v_ccc: 0.2745 - a_ccc: 0.6392 - d_ccc: 0.

6295/6295 [==============================] - 3s 522us/step - loss: 1.4386 - v_loss: 0.6254 - a_loss: 0.3523 - d_loss: 0.4606 - v_ccc: 0.3744 - a_ccc: 0.6477 - d_ccc: 0.5393 - val_loss: 1.9908 - val_v_loss: 0.8466 - val_a_loss: 0.5352 - val_d_loss: 0.6026 - val_v_ccc: 0.1506 - val_a_ccc: 0.4637 - val_d_ccc: 0.3950
Epoch 13/50
2170/2170 [==============================] - 0s 168us/step
[0.09899197518825531, 0.5722887516021729, 0.44677308201789856]
6
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 9s 1ms/step - loss: 1.9450 - v_loss: 0.9755 - a_loss: 0.4155 - d_loss: 0.5544 - v_ccc: 0.0249 - a_ccc: 0.5843 - d_ccc: 0.4458 - val_loss: 2.4649 - val_v_loss: 0.9373 - val_a_loss: 0.7753 - val_d_loss: 0.7466 - val_v_ccc: 0.0623 - val_a_ccc: 0.2239 - val_d_ccc: 0.2489
Epoch 2/50
6295/6295 [==============================] - 3s 522us/step - loss: 1.6192 - v_loss: 0.7832 - a_loss: 0.3610 - d_loss: 0.4746 - v_ccc: 0.2166 - a_ccc: 0.6390 - d_ccc: 

...

Epoch 19/50
6295/6295 [==============================] - 3s 517us/step - loss: 1.3166 - v_loss: 0.5243 - a_loss: 0.3478 - d_loss: 0.4448 - v_ccc: 0.4760 - a_ccc: 0.6521 - d_ccc: 0.5553 - val_loss: 1.9937 - val_v_loss: 0.8362 - val_a_loss: 0.5341 - val_d_loss: 0.6169 - val_v_ccc: 0.1573 - val_a_ccc: 0.4677 - val_d_ccc: 0.3813
Epoch 20/50
6295/6295 [==============================] - 3s 549us/step - loss: 1.3198 - v_loss: 0.5277 - a_loss: 0.3472 - d_loss: 0.4447 - v_ccc: 0.4723 - a_ccc: 0.6526 - d_ccc: 0.5552 - val_loss: 2.0490 - val_v_loss: 0.8427 - val_a_loss: 0.5639 - val_d_loss: 0.6426 - val_v_ccc: 0.1545 - val_a_ccc: 0.4373 - val_d_ccc: 0.3592
Epoch 21/50
2170/2170 [==============================] - 0s 212us/step
[0.11829859018325806, 0.5555270910263062, 0.4187904894351959]
13
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 13s 2ms/step - loss: 1.9172 - v_loss: 0.9535 - a_loss: 0.4070 - d_loss: 0.5570 - v_ccc: 0.0465 - a_ccc: 0.

6295/6295 [==============================] - 4s 563us/step - loss: 1.6599 - v_loss: 0.8265 - a_loss: 0.3543 - d_loss: 0.4794 - v_ccc: 0.1736 - a_ccc: 0.6458 - d_ccc: 0.5207 - val_loss: 1.9817 - val_v_loss: 0.8609 - val_a_loss: 0.5423 - val_d_loss: 0.5827 - val_v_ccc: 0.1388 - val_a_ccc: 0.4615 - val_d_ccc: 0.4180
Epoch 3/50
6295/6295 [==============================] - 4s 559us/step - loss: 1.5622 - v_loss: 0.7362 - a_loss: 0.3556 - d_loss: 0.4707 - v_ccc: 0.2642 - a_ccc: 0.6444 - d_ccc: 0.5292 - val_loss: 1.9483 - val_v_loss: 0.8552 - val_a_loss: 0.5133 - val_d_loss: 0.5842 - val_v_ccc: 0.1434 - val_a_ccc: 0.4920 - val_d_ccc: 0.4163
Epoch 4/50
6295/6295 [==============================] - 4s 587us/step - loss: 1.5407 - v_loss: 0.7001 - a_loss: 0.3673 - d_loss: 0.4731 - v_ccc: 0.2996 - a_ccc: 0.6327 - d_ccc: 0.5270 - val_loss: 1.9872 - val_v_loss: 0.8564 - val_a_loss: 0.5273 - val_d_loss: 0.6066 - val_v_ccc: 0.1417 - val_a_ccc: 0.4763 - val_d_ccc: 0.3948
Epoch 5/50
6295/6295 [===========

Epoch 14/50
2170/2170 [==============================] - 0s 217us/step
[0.10902446508407593, 0.5811012983322144, 0.44614362716674805]
16
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 14s 2ms/step - loss: 1.9085 - v_loss: 0.9338 - a_loss: 0.4280 - d_loss: 0.5475 - v_ccc: 0.0661 - a_ccc: 0.5724 - d_ccc: 0.4531 - val_loss: 2.5381 - val_v_loss: 0.8743 - val_a_loss: 0.8404 - val_d_loss: 0.8168 - val_v_ccc: 0.1238 - val_a_ccc: 0.1585 - val_d_ccc: 0.1796
Epoch 2/50
6295/6295 [==============================] - 4s 589us/step - loss: 1.6049 - v_loss: 0.7575 - a_loss: 0.3696 - d_loss: 0.4782 - v_ccc: 0.2425 - a_ccc: 0.6304 - d_ccc: 0.5222 - val_loss: 2.0598 - val_v_loss: 0.8963 - val_a_loss: 0.5744 - val_d_loss: 0.5953 - val_v_ccc: 0.1037 - val_a_ccc: 0.4301 - val_d_ccc: 0.4064
Epoch 3/50
6295/6295 [==============================] - 4s 575us/step - loss: 1.5538 - v_loss: 0.7169 - a_loss: 0.3649 - d_loss: 0.4724 - v_ccc: 0.2831 - a_ccc: 0.6

6295/6295 [==============================] - 4s 576us/step - loss: 1.5838 - v_loss: 0.7522 - a_loss: 0.3568 - d_loss: 0.4744 - v_ccc: 0.2476 - a_ccc: 0.6432 - d_ccc: 0.5254 - val_loss: 1.9740 - val_v_loss: 0.8629 - val_a_loss: 0.5188 - val_d_loss: 0.5943 - val_v_ccc: 0.1356 - val_a_ccc: 0.4847 - val_d_ccc: 0.4058
Epoch 4/50
6295/6295 [==============================] - 4s 598us/step - loss: 1.5597 - v_loss: 0.7237 - a_loss: 0.3616 - d_loss: 0.4739 - v_ccc: 0.2762 - a_ccc: 0.6383 - d_ccc: 0.5257 - val_loss: 1.9705 - val_v_loss: 0.8683 - val_a_loss: 0.5080 - val_d_loss: 0.5954 - val_v_ccc: 0.1297 - val_a_ccc: 0.4957 - val_d_ccc: 0.4042
Epoch 5/50
6295/6295 [==============================] - 4s 592us/step - loss: 1.5381 - v_loss: 0.7038 - a_loss: 0.3621 - d_loss: 0.4721 - v_ccc: 0.2962 - a_ccc: 0.6377 - d_ccc: 0.5280 - val_loss: 2.0255 - val_v_loss: 0.8688 - val_a_loss: 0.5482 - val_d_loss: 0.6067 - val_v_ccc: 0.1294 - val_a_ccc: 0.4533 - val_d_ccc: 0.3918
Epoch 6/50
6295/6295 [===========

Epoch 14/50
6295/6295 [==============================] - 4s 622us/step - loss: 1.3874 - v_loss: 0.5876 - a_loss: 0.3497 - d_loss: 0.4498 - v_ccc: 0.4123 - a_ccc: 0.6503 - d_ccc: 0.5500 - val_loss: 2.0979 - val_v_loss: 0.8559 - val_a_loss: 0.6072 - val_d_loss: 0.6383 - val_v_ccc: 0.1418 - val_a_ccc: 0.3972 - val_d_ccc: 0.3632
Epoch 15/50
6295/6295 [==============================] - 4s 643us/step - loss: 1.3760 - v_loss: 0.5772 - a_loss: 0.3503 - d_loss: 0.4487 - v_ccc: 0.4230 - a_ccc: 0.6499 - d_ccc: 0.5511 - val_loss: 1.9878 - val_v_loss: 0.8472 - val_a_loss: 0.5393 - val_d_loss: 0.6065 - val_v_ccc: 0.1522 - val_a_ccc: 0.4659 - val_d_ccc: 0.3941
Epoch 16/50
6295/6295 [==============================] - 4s 626us/step - loss: 1.3796 - v_loss: 0.5869 - a_loss: 0.3438 - d_loss: 0.4480 - v_ccc: 0.4126 - a_ccc: 0.6561 - d_ccc: 0.5516 - val_loss: 2.0075 - val_v_loss: 0.8484 - val_a_loss: 0.5426 - val_d_loss: 0.6180 - val_v_ccc: 0.1502 - val_a_ccc: 0.4611 - val_d_ccc: 0.3812
Epoch 17/50
6295/62